In [30]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import pandas as pd

def carregar_dados():
    df = pd.read_excel('notas_estudantes.xlsx', sheet_name='Dados')
    df['Média'] = df[['Nota 1','Nota 2','Nota 3', 'Nota 4']].mean(axis=1)
    df['Situação'] = df.apply(lambda row: classificar_situacao(row['Média'],row['Faltas']),axis=1)
    return df

def classificar_situacao(media, faltas):
    if faltas > 10:
        return 'Reprovado por Faltas'
    
    elif media < 2:
        return 'Reprovado por Nota'
    
    elif 2 <= media < 7:
        return 'Recuperação'
    
    else:
        return 'Aprovado'

def calcular_estatisticas(df):
    estatisticas = df.groupby('Turma').agg(
        Media_da_Turma = ('Média', 'mean'),
        Total_de_Faltas = ('Faltas', 'sum')
    ).reset_index()

    estatisticas = estatisticas.sort_values(by='Turma')

    return estatisticas

def filtrar_alunos(*args):
    filtro = entrada_filtro.get().lower()
    alunos_Filtrados_locais = alunos_Filtrados[alunos_Filtrados['Nome'].str.lower().str.contains(filtro) |
                                               alunos_Filtrados['Situação'].str.lower().str.contains(filtro)]

    for item in tabela_detalhes.get_children():
        tabela_detalhes.delete(item)

    for _, aluno in alunos_Filtrados_locais.iterrows():
        tabela_detalhes.insert('','end',
                               values=(aluno['Nome'],
                                       f"{aluno['Média']:.2f}",
                                       aluno['Situação']))    

def exportar_para_excel():
    alunos_Filtrados_loacais = alunos_Filtrados[alunos_Filtrados['Nome'].str.lower().str.contains(entrada_filtro.get().lower()) |
                                                alunos_Filtrados['Situação'].str.lower().str.contains(entrada_filtro.get().lower())]
    
    if not alunos_Filtrados_loacais.empty:
        caminho_arquivo = filedialog.asksaveasfilename(defaultextension=".xlsx",filetypes=[("Excel files","*.xlsx")])

        if caminho_arquivo:
            alunos_Filtrados_loacais.to_excel(caminho_arquivo, index=False)
            messagebox.showinfo("Exportação bem sucedida", f"Os dados foram exportados para {caminho_arquivo} com sucesso.")

def exibir_detalhes(evento):
    linha_id = tabela.identify_row(evento.y)

    if linha_id:
        item = tabela.item(linha_id)
        valores = item['values']
        turma = valores[0]
        
        global alunos_Filtrados
        alunos_Filtrados = df[df['Turma'] == turma]

        atualizar_tabela_detalhes()

        titulo_turma.config(text=f"Turma: {turma}")

def atualizar_tabela_detalhes():
    for item in tabela_detalhes.get_children():
        tabela_detalhes.delete(item)

    for _, aluno in alunos_Filtrados.iterrows():
        tabela_detalhes.insert('', 'end', 
                               values=(aluno['Nome'], 
                                       f"{aluno['Média']:.2f}",
                                       aluno['Situação']))

def exibir_tootip_aluno(evento):
    linha_id = tabela_detalhes.identify_row(evento.y)

    if linha_id:
        item = tabela_detalhes.item(linha_id)
        nome_aluno = item['values'][0]
        aluno = df[df['Nome'] == nome_aluno].iloc[0]
        texto_aluno = (f"Nome: {aluno['Nome']}\n"
                       f"Nota 1: {aluno['Nota 1']}\n"
                       f"Nota 2: {aluno['Nota 2']}\n"
                       f"Nota 3: {aluno['Nota 3']}\n"
                       f"Nota 4: {aluno['Nota 4']}\n"
                       f"Média: {aluno['Média']}\n"
                       f"Faltas: {aluno['Faltas']}\n"
                       f"Situação: {aluno['Situação']}\n")
        
        label_tooltip_aluno.config(text=texto_aluno)

        tooltip_aluno.wm_geometry(f"+{evento.x_root + 10}+{evento.y_root + 10}")

        tooltip_aluno.deiconify()

def ocultar_tootip_aluno(evento):
    tooltip_aluno.withdraw()

janela = tk.Tk()
janela.title("Resumo de Turmas")

df = carregar_dados()

estatisticas = calcular_estatisticas(df)

fonte_padrao = ("Arial", 12)
font_detalhes = ("Arial", 12, "bold")

colunas = ('Turma', 'Média da Turma', 'Total de Faltas')

tabela = ttk.Treeview(janela,
                      columns=colunas,
                      show='headings',
                      style='mystyle.Treeview')
tabela.pack(side='left', fill='both',expand=True)

for col in colunas:
    tabela.heading(col, text=col)

painel_lateral = tk.Frame(janela,
                          bg='lightyellow',
                          relief='solid',
                          borderwidth=2,
                          width=300)
painel_lateral.pack(side='right',fill='y')

titulo_turma = tk.Label(painel_lateral,
                        text='Selecione uma turma',
                        font=font_detalhes,
                        bg='lightyellow')
titulo_turma.pack(padx=10,pady=10)

entrada_filtro = tk.Entry(painel_lateral, font=fonte_padrao)
entrada_filtro.pack(fill='x', padx=10,pady=5)
entrada_filtro.bind('<KeyRelease>', filtrar_alunos)

botao_exportar = tk.Button(painel_lateral,
                           text="Exportar para Excel",
                           command=exportar_para_excel,
                           font=fonte_padrao)
botao_exportar.pack(padx=10,pady=5)

tabela_detalhes = ttk.Treeview(painel_lateral,
                               columns=("Nome","Média","Situação"),
                               show='headings',
                               style='mystyle.Treeview')
tabela_detalhes.pack(fill='both',expand=True)

for col in ('Nome','Média','Situação'):
    tabela_detalhes.heading(col, text=col)

tooltip_aluno = tk.Toplevel(janela)
tooltip_aluno.withdraw()
tooltip_aluno.overrideredirect(True)

label_tooltip_aluno = tk.Label(tooltip_aluno,
                               text="",
                               justify='left',
                               background='lightblue',
                               font=font_detalhes)
label_tooltip_aluno.pack(padx=10,pady=5)

tabela.bind('<Motion>', exibir_detalhes)
tabela_detalhes.bind('<Motion>', exibir_tootip_aluno)
tabela_detalhes.bind('<Leave>', ocultar_tootip_aluno)

for _, row in estatisticas.iterrows():
    tabela.insert('', 'end', 
                  values=(row['Turma'], 
                          f"{row['Media_da_Turma']:.2f}",
                          row['Total_de_Faltas']))

carregar_dados()

janela.mainloop()